# DEPENDÊNCIAS

In [ ]:
! pip install spacy
! pip install nltk
! pip install pandas
! pip install pdfplumber
! pip install chromadb
! pip install sentence-transformers
! pip install openpyxl
! pip install python-docx
! python -m spacy download pt_core_news_sm
! pip install langchain-text-splitters

# Carrega PDF

In [9]:
import pdfplumber

pdf_path = "./documents/sample_portugues.pdf"

def load_pdf(filepath):
    with pdfplumber.open(filepath) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

raw_text = load_pdf(pdf_path)

print("Prévia do texto lido:\n")
print(raw_text[:800])


Prévia do texto lido:

Título: Introdução ao Machine Learning
Machine learning é um campo da inteligência artificial que desenvolve algoritmos
capazes de aprender padrões a partir de dados. Os principais tipos incluem
aprendizado supervisionado, aprendizado não supervisionado e aprendizado por
reforço.
Aprendizado supervisionado envolve treinar um modelo com dados rotulados,
como prever preços de casas com base em características como tamanho,
localização e número de quartos. Aprendizado não supervisionado detecta
padrões ocultos em dados não rotulados, como segmentação de clientes em
marketing. Aprendizado por reforço ensina agentes a tomar decisões em
ambientes dinâmicos para maximizar recompensas.
Redes neurais profundas são usadas em visão computacional e processamento
de linguagem natural. Modelos como CNNs


# PLN

## Lowercasing

In [10]:
text_lower = raw_text.lower()

print("Texto em lowercase (prévia):\n")
print(text_lower[:800])


Texto em lowercase (prévia):

título: introdução ao machine learning
machine learning é um campo da inteligência artificial que desenvolve algoritmos
capazes de aprender padrões a partir de dados. os principais tipos incluem
aprendizado supervisionado, aprendizado não supervisionado e aprendizado por
reforço.
aprendizado supervisionado envolve treinar um modelo com dados rotulados,
como prever preços de casas com base em características como tamanho,
localização e número de quartos. aprendizado não supervisionado detecta
padrões ocultos em dados não rotulados, como segmentação de clientes em
marketing. aprendizado por reforço ensina agentes a tomar decisões em
ambientes dinâmicos para maximizar recompensas.
redes neurais profundas são usadas em visão computacional e processamento
de linguagem natural. modelos como cnns


## Remover caracteres especiais

In [ ]:
import re
import unicodedata

def remove_special_chars(text):
    # Normaliza acentos (NFKD)
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore").decode("utf-8")

    # Remove caracteres que não sejam letras ou espaço
    text = re.sub(r"[^a-z0-9.,\s]", " ", text)

    # Remove múltiplos espaços
    text = re.sub(r"\s+", " ", text)

    return text.strip()

text_clean = remove_special_chars(text_lower)

print("Texto sem caracteres especiais (prévia):\n")
print(text_clean[:800])


Texto sem caracteres especiais (prévia):

titulo introducao ao machine learning machine learning e um campo da inteligencia artificial que desenvolve algoritmos capazes de aprender padroes a partir de dados. os principais tipos incluem aprendizado supervisionado, aprendizado nao supervisionado e aprendizado por reforco. aprendizado supervisionado envolve treinar um modelo com dados rotulados, como prever precos de casas com base em caracteristicas como tamanho, localizacao e numero de quartos. aprendizado nao supervisionado detecta padroes ocultos em dados nao rotulados, como segmentacao de clientes em marketing. aprendizado por reforco ensina agentes a tomar decisoes em ambientes dinamicos para maximizar recompensas. redes neurais profundas sao usadas em visao computacional e processamento de linguagem natural. modelos como cnns 


## Remover Stopwords

In [72]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Baixar recursos necessários (roda apenas 1 vez)
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

stopwords_pt = set(stopwords.words("portuguese"))

tokens = word_tokenize(text_clean, language="portuguese")

tokens_no_stop = [t for t in tokens if t not in stopwords_pt]

print("Tokens sem stopwords (primeiros 30):\n")
print(tokens_no_stop[:30])


Tokens sem stopwords (primeiros 30):

['titulo', 'introducao', 'machine', 'learning', 'machine', 'learning', 'campo', 'inteligencia', 'artificial', 'desenvolve', 'algoritmos', 'capazes', 'aprender', 'padroes', 'partir', 'dados', '.', 'principais', 'tipos', 'incluem', 'aprendizado', 'supervisionado', ',', 'aprendizado', 'nao', 'supervisionado', 'aprendizado', 'reforco', '.', 'aprendizado']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Lemmatização

In [79]:
import spacy

# Carregar modelo de português
nlp = spacy.load("pt_core_news_sm")

def lemmatize_text(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

lemmas = " ".join(lemmatize_text(tokens_no_stop))

print("Lemmas (primeiros 30):\n")
print(lemmas[:300])


Lemmas (primeiros 30):

titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado . principal tipo incluir aprendizar supervisionar , aprendizar nao supervisionar aprendizar reforco . aprendizar supervisionar envolver treinar modelo dado rotul


## Stemmatização

In [80]:
# --- STEMMATIZAÇÃO (alternativa à lematização) ---
from nltk.stem.snowball import SnowballStemmer

# Criar stemmer para português
stemmer = SnowballStemmer("portuguese")

# Aplicar stemmatização nos tokens sem stopwords
stems = " ".join([stemmer.stem(token) for token in tokens_no_stop])

print("Stems (primeiros 30):\n")
print(stems[:300])


Stems (primeiros 30):

titul introduca machin learning machin learning camp inteligenc artificial desenvolv algoritm capaz aprend padro part dad . princip tip inclu aprendiz supervision , aprendiz nao supervision aprendiz reforc . aprendiz supervision envolv trein model dad rotul , prev prec cas bas caracterist tamanh , l


# Chunking


### Por Tamanho Fixo

In [119]:
# --- CHARACTER TEXT SPLITTER ---
from langchain_text_splitters import CharacterTextSplitter

char_splitter = CharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=10,
    separator=".",
    length_function=len
)

chunks_char = char_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_char)}")
print(chunks_char)


Total de chunks: 7
['titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado', 'principal tipo incluir aprendizar supervisionar , aprendizar nao supervisionar aprendizar reforco', 'aprendizar supervisionar envolver treinar modelo dado rotular , prever preco casa base caracteristico tamanho , localizacaar numero quarto', 'aprendizar nao supervisionar detecta padroe oculto dar nao rotular , segmentacao cliente Marketing', 'aprendizar reforco ensina agente tomar decisoes ambiente dinamico maximizar recompensa', 'rede neural profundo sao usar visao computacional processamento linguagem natural', 'modelo cnns transformer revolucionar tarefa traducao automatica , reconhecimento imagem geracaar texto']


### Recursívo

In [121]:
# --- RECURSIVE CHARACTER TEXT SPLITTER ---
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=10,
    separators=["\n\n", "\n", ".", ",", " ", ""]
)

chunks_recursive = recursive_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_recursive)}")
print(chunks_recursive)


Total de chunks: 7
['titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado', '. principal tipo incluir aprendizar supervisionar , aprendizar nao supervisionar aprendizar reforco', '. aprendizar supervisionar envolver treinar modelo dado rotular , prever preco casa base caracteristico tamanho , localizacaar numero quarto', '. aprendizar nao supervisionar detecta padroe oculto dar nao rotular , segmentacao cliente Marketing', '. aprendizar reforco ensina agente tomar decisoes ambiente dinamico maximizar recompensa', '. rede neural profundo sao usar visao computacional processamento linguagem natural', '. modelo cnns transformer revolucionar tarefa traducao automatica , reconhecimento imagem geracaar texto .']


# Embedding

## Hugging Face SentenceTransformers

In [152]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("rufimelo/bert-base-portuguese-cased-sts")

def embed_hf(chunks):
    vectors = model.encode(chunks, show_progress_bar=True)

    print(f"Dimensão: {vectors.shape}")
    print(vectors[:5])

    return vectors

# Testes
embedding = embed_hf(chunks_recursive)


Invalid model-index. Not loading eval results into CardData.
Batches: 100%|██████████| 1/1 [00:00<00:00,  2.11it/s]

Dimensão: (7, 1024)
[[-0.49964434 -0.7541532  -0.7782051  ...  1.1481283  -0.9186344
  -0.340363  ]
 [ 0.1388875  -0.21344706 -0.836119   ...  0.5647964  -0.5228001
   0.68648016]
 [-0.05482586 -0.02724688  0.2068215  ...  0.28013238 -0.21512774
   0.49167874]
 [-0.605365   -0.33865842 -0.12960064 ...  0.6480235  -0.20835035
  -0.18436617]
 [-0.19067355  0.0421397   0.3545091  ...  0.11871685 -0.74754953
  -0.2962598 ]]


# Banco Vetorial

## Criação/Inserção ChromaDb

In [154]:
import chromadb

client = chromadb.Client()

collection = client.get_or_create_collection(
    name="exemplo_colecao",
    embedding_function=None,
    # metadata={"hnsw:space": "cosine"}   # métrica mais comum
)
print(f"{collection.count()} documentos na coleção")

0 documentos na coleção


In [155]:
ids = [f"chunk_{i}" for i in range(len(chunks_recursive))]

collection.add(
    ids=ids,
    documents=chunks_recursive,
    embeddings=embedding
)

print("Embeddings inseridos com sucesso!")


Embeddings inseridos com sucesso!


## Busca Semântica

In [158]:
question = "Quais são os tipos de aprendizado de máquina?"
question_embedding = model.encode([question])

resultados = collection.query(
    query_embeddings=question_embedding.tolist(),
    n_results=3
)

print(f"\nPergunta: {question}")

for doc, score in zip(resultados['documents'][0], resultados['distances'][0]):
    print(f"(Score: {score}) - Documento: {doc}")


Pergunta: Quais são os tipos de aprendizado de máquina?
(Score: 379.6021728515625) - Documento: titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado
(Score: 452.904052734375) - Documento: . rede neural profundo sao usar visao computacional processamento linguagem natural
(Score: 490.55108642578125) - Documento: . modelo cnns transformer revolucionar tarefa traducao automatica , reconhecimento imagem geracaar texto .
